## import 导入链的注意事项

导入是将被导入模块中的名称添加到导入模块的命名空间中的过程。

假设当前模块名为 `M`，`from X import Xo` 会将 `Y` 直接添加到当前模块的命名空间：可以访问 `M.Xo`

而 `import X` 会将整个模块 `X` 作为一个对象添加到当前模块的命名空间：`M.X`（包括例如 `M.X.Xo`）等。

例子 1：

```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
from A import value, func  # 从A导入特定内容到B的命名空间

# 模块C (c.py)
import B

# 在C中可以使用:
print(B.value)  # 可以访问，输出100
print(B.func())  # 可以访问，输出"A.func"
```
例子 2：
```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
import A  # A成为B模块命名空间中的一个对象

# 模块C (c.py)
import B

# 在C中:
print(B.A.value)  # 可以访问，输出100
print(B.value)    # 错误! B中没有直接定义value

## 描述符
### 描述符协议

描述符协议规定了：`描述符` 作为某个属主类的 `类属性` 时，属主类使用该描述符时实际发生的行为。

一个类要成为 `描述符`，至少要实现下面三个特殊方法中的一个：

```python
__get__(self, instance, owner)
__set__(self, instance, value)
__delete__(self, instance)
```

其中 `instance` 是属主类实例，`owner` 是属主类本身。

```python
class Descriptor:

    def __init__(self):
        ...

    def __get__(self, instance, owner):
        ...

    def __set__(self, instance, value):
        ...

    def __delete__(self, instance):
        ...

class User:
    descriptor = Descriptor()

user = User()
```

当类似于 `= user.descriptor` 使用时，实际上是触发了 `descriptor.__get__(user, User)`。

`user.descriptor = 值` 实际上触发了 `descriptor.__set__(user, 值)`。

`del user.descriptor` 实际上触发了 `descriptor.__delete__(user)`。

### property 类

`property` 是一个实现了描述符协议的 Python 内置类，其实现：

```python
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc"

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    # 返回新的 Property 实例
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

### @property

使用方法如下

```python
class MyClass:
    def __init__(self):

    # value = property(函数 value(self))
    @property
    def value(self):
        ...

    # value = value.setter(函数 value(self, new_value))
    @value.setter
    def value(self, new_value):
        ...

    # value = value.setter(函数 value(self))
    @value.deleter
    def value(self):
        ...
```

所以相当于创建了三次 property 实例，最后该实例成为 MyClass 的一个类属性 value。

然后 MyClass 或其实例就可以按照 `描述符协议` 来使用 value。

## `with open(str, '*') as f:`
等价于
```python
try:
    f = open(str, '*')
    ...
finally:
    if f:
        f.close()
```

## `__new__(cls, *args, **kwargs)`

### 类的方法定义中的 `self` 和 `cls`
```python
class MyClass:
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)

    @classmethod
    def funcClass(cls, *args, **kwargs):...

    def func(self, *args, **kwargs):...

instance = MyClass()
instance.func()
```
当 `instance.func()` 时，实例方法会自动将实例 `instance` 作为第一个参数 `self`。

当 `instance.funcClass()` 或 `MyClass.funcClass()` 时，类方法会识别出类型作为第一个参数 `cls`。

至于为什么一般写为 `self` 和 `cls`，这只是约定俗成的惯例。

### `__new__(cls, *args, **kwargs)` 是静态方法
`__new__` 主要作用是分配内存并创建实例本身，其一般是通知父类的 `__new__` 创建对应类型的实例：
```python
def __new__(cls, *args, **kwargs):
    return super().__new__(cls)
```
如果父类直接继承自 `object` 或者没有重定义 `__new__`，那么最终实际上是 `object` 调用其 `__new__` 创建对应类型的实例。

由于 `__new__(cls, *args, **kwargs)` 不是类方法，所以 `cls` 是手动传入的，其作用在于告诉 `object` 要创建什么类型的实例。
### 类型创建实例的过程
当一个类型创建实例时，例如 `instance = MyClass()`，Python 内部实际执行
```python
instance = MyClass.__new__(MyClass)
if isinstance(instance, MyClass):
    instance.__init__()
```
`instance = MyClass.__new__(MyClass)` 这一行相当于告诉 `object` 创建一个 MyClass 类型的实例，而 `instance.__init__()` 负责初始化。

## Python 类型和实例的内存构成
### 实例
`__dict__` 字典的引用：存储实例在运行时动态添加或在 `__init__` 方法中设置的属性。
- 如果一个类定义了 `__slots__`，并且其中没有包含 `__dict__`，那么该类的实例将不会有 `__dict__` 属性。这种情况下，实例属性（名称包含在 `__slots__` 中）在内存中是独立存储的
- 许多内置类型的实例（如 list, tuple, str, int, set, map 对象，甚至 dict 实例本身）为了效率和设计目的，没有 `__dict__` 属性，属性直接存储在固定内存布局中

`__class__` 引用：指向实例所属的类对象（如 `<class '__main__.A'>`）

`__weakref__`：支持弱引用机制，仅在需要时存在。
### 类型
`__dict__` 字典的引用：存储类属性、方法、描述符等。

`__name__`：类的名称。

`__module__`：类定义所在的模块名。

`__bases__`：类的父类元组

`__doc__`：类的文档字符串。

`__qualname__ `：类的限定名称（包含嵌套类路径）。

`方法表`：存储类的方法（如 `__init__`, `__str__`）。

## 递归

递归函数是任意的，只要一个函数的实现中调用了自身。

最简单的情况
- 在域 `D` 上完成某功能，是否能将其转换为：在 `D` 的不相交子域上各自完成的功能然后整合而成。

另一种常见情况
- 因为域 `D` 构成过于复杂，事先甚至无法将在域 `D` 上的要实现的功能完备描述出来。
- 可以先考虑域 `D` 的最简单情况：可以直接解决而无须递归。然后逐步考虑完整的情况。

很多问题无法运用套路，但是仍可能通过递归解决，需要具体分析。

## 实例的内存理解
`域` 即一段内存空间
- 每个域具有 `域类型`
- 域分为`值域`和`引用域`
    - 值域存储的是实际内容 `域名.字段名 = 值`
    - 引用域存储的是另一个域的地址 `域名.字段名 = 域名`
- 对于一个域 $o$，从其引用域延申到其它域，这样一直延申，可以得到一个分层次的、由多个域构成、多分支有向连接的 `图`
    - 当然可能有其它域的字段引用 `o`，但是这里只考虑从 `o` 出发延申得到的
    - 图上的两个域 $a$ 和 $b$，如果 $a.x=b$，则称有一条 $a\mathop  \to \limits^{a.x} b$ 的`路径`。
        - 如果存在形如 $a \to  \cdots  \to a$ 的路径，则称该图是 `有环图`
    - 如何不重复的遍历图中的所有域：
        ```python
        def Vist(域x, visited):
            if visited.contains(id(x))
                return
            visited[id(x)] = True
            对于 x.引用域中的每个引用i：
                Vist(i, visited)
        ```

## 浅拷贝和深拷贝
### 浅拷贝
将被拷贝实例的域中的内容完整的拷贝到另一段内存空间。所以两段内存空间中的引用域指向相同的实例。
### 深拷贝
深拷贝就是在内存中开辟另一个图，使得值域、对应有向连接和原图相同。

假设有一个函数 `deepcopy(实例x, memo)`，返回深拷贝实例x后的地址，其伪代码如下：
```python
def deepcopy(实例x, memo):
    if memo.contains(id(x))
        return memo[id(x)]
    内存中创建一个和x同类型的实例y
    memo[id(x)] = y
    y.值域 = x.值域
    遍历 x.引用域中的每个引用i：
        y.i = deepcopy(引用i，memo)
    return y
```

## 代码分析
### 类继承关系
```mermaid
classDiagram
    %% 基本类
    class object
    
    %% 一级继承
    class dict
    class Default
    class Pickleable
    class Documented
    class tuple
    
    %% 二级继承
    class atomic_dict
    class PickleableDict
    class Configured
    class namedtuple
    
    %% 三级继承
    class Config
    class DumpTuple
    
    %% 四级继承
    class AtomicConfig
    
    %% 所有类都继承自 object
    object <|-- dict
    object <|-- Default
    object <|-- Pickleable
    object <|-- Documented
    object <|-- tuple
    
    %% 其他继承关系
    dict <|-- atomic_dict
    tuple <|-- namedtuple
    
    Pickleable <|-- PickleableDict
    dict <|-- PickleableDict
    
    Pickleable <|-- Configured
    Documented <|-- Configured
    
    PickleableDict <|-- Config
    Documented <|-- Config
    
    Config <|-- AtomicConfig
    atomic_dict <|-- AtomicConfig
    
    namedtuple <|-- DumpTuple
```
`atomic_dict` 和 `Config` 都是 `dict` 的子类，它们三个类的 `items()` 都是用来存储`配置参数`的。

`Config` 的 `__dict__` 用来存储 `如何管理配置参数的参数`。但是，也可以通过 `_as_attrs_` 来将`配置参数`同时存储到 `__dict__`。

### 递归设计：`子图`

考虑 `dict` 类型（`dict/atomic_dict/Config`）的实例出发构成的图。

如果在延申过程中，对于 `dict` 类型域只考虑其 `items()` 字段（例如 `Config` 不考虑 `__dict__`），那么后者构成的图可以理解为原图的 `dict子图`。


#### 函数 `convert_to_dict`
```python
def convert_to_dict(dct: InConfigLikeT, nested: bool = True) -> dict:
    if dct is None:
        dct = {}
    if isinstance(dct, atomic_dict):
        dct = atomic_dict(dct)
    else:
        dct = dict(dct) 
    if not nested:
        return dct
    for k, v in dct.items():
        if isinstance(v, dict):
            dct[k] = convert_to_dict(v, nested=nested)
        else:
            dct[k] = v
    return dct
```

返回 `dct` 的子图，且其中除了 atomic_dict 的 dict 类型的域都被转换成了纯 dict 类型（`nested` 决定考虑所有域还是只有直接域）。

#### 函数 `update_dict`
```python
def update_dict(x: InConfigLikeT,
                y: InConfigLikeT,
                nested: bool = True,
                force: bool = False,
                same_keys: bool = False) -> None:
    if x is None:
        return
    if y is None:
        return
    checks.assert_instance_of(x, dict)
    checks.assert_instance_of(y, dict)
    for k, v in y.items():
        if nested \
                and k in x \
                and isinstance(x[k], dict) \
                and isinstance(v, dict) \
                and not isinstance(v, atomic_dict):
            update_dict(x[k], v, force=force)
        else:
            if same_keys and k not in x:
                continue
            set_dict_item(x, k, v, force=force)
```

用 `y` 的子图对应更新（去尾路径字段名相同，且对应 `y` 的域不能是 atomic_dict 类型） `x` 的子图（`nested` 决定考虑所有域还是只有直接域）：
- 两字段重名，则用 `y` 对应的字段直接替换 `x` 对应的字段。
- 如果 `same_keys` 关闭且 `y`对应域的某字段在 `x` 对应域中不存在，则用 `y` 对应域的字段值在 `x` 对应域中生成该字段。

#### 函数 `Config.__copy__` 和 `Config.__deepcopy__`

如果 `self.as_attrs_ == true`，说明 `配置参数` 同时存到了 `__dict__` 中。
```python
def  _clear_attrs(self, prior_keys: tp.Iterable[str]) -> None:
    if self.as_attrs_:
        for k in set(prior_keys).difference(self.keys()):
            del self.__dict__[k]
```

```python
def clear(self, force: bool = False) -> None:
    if not force and self.readonly_:
        raise TypeError("Config is read-only")
    if not force and self.frozen_keys_:
        raise KeyError(f"Config keys are frozen")
    prior_keys = list(self.keys())
    dict.clear(self)
    self._clear_attrs(prior_keys)
```

```python
def __copy__(self: ConfigT) -> ConfigT:
    cls = self.__class__
    self_copy = cls.__new__(cls)
    for k, v in self.__dict__.items():
        if k not in self_copy:  
            self_copy.__dict__[k] = v
    self_copy.clear(force=True)
    self_copy.update(copy(dict(self)), nested=False, force=True)
    return self_copy
```

```python
def __deepcopy__(self: ConfigT, memo: tp.DictLike = None) -> ConfigT:
    if memo is None:
        memo = {}
    cls = self.__class__
    self_copy = cls.__new__(cls)
    memo[id(self)] = self_copy
    for k, v in self.__dict__.items():
        if k not in self_copy:  
            self_copy.__dict__[k] = deepcopy(v, memo)
    self_copy.clear(force=True)
    self_copy.update(deepcopy(dict(self), memo), nested=False, force=True)
    return self_copy
```

#### 函数 `copy_dict` 和 `Config.copy`
```python
def copy_dict(dct: InConfigLikeT, copy_mode: str = 'shallow', nested: bool = True) -> OutConfigLikeT:
    if dct is None:
        dct = {}
    checks.assert_instance_of(copy_mode, str)
    copy_mode = copy_mode.lower()
    if copy_mode not in ['shallow', 'hybrid', 'deep']:
        raise ValueError(f"Copy mode '{copy_mode}' not supported")

    if copy_mode == 'deep':
        return deepcopy(dct)
    
    if isinstance(dct, Config):
        return dct.copy(
            copy_mode=copy_mode,
            nested=nested
        )
        
    dct_copy = copy(dct) 
    for k, v in dct_copy.items():
        if nested and isinstance(v, dict):
            _v = copy_dict(v, copy_mode=copy_mode, nested=nested)
        else:
            if copy_mode == 'hybrid':
                _v = copy(v)  
            else:
                _v = v
        set_dict_item(dct_copy, k, _v, force=True)
    return dct_copy

def copy(self: ConfigT, reset_dct_copy_kwargs: tp.KwargsLike = None, **copy_kwargs) -> ConfigT:
    self_copy = self.__copy__()
    
    reset_dct_copy_kwargs = merge_dicts(self.reset_dct_copy_kwargs_, copy_kwargs, reset_dct_copy_kwargs)
    reset_dct = copy_dict(dict(self.reset_dct_), **reset_dct_copy_kwargs)
    self.__dict__['_reset_dct_'] = reset_dct

    copy_kwargs = merge_dicts(self.copy_kwargs_, copy_kwargs)
    dct = copy_dict(dict(self), **copy_kwargs)
    self_copy.update(dct, nested=False, force=True)

    return self_copy
```

先考虑最简单的情况：
- （1） `dct` 为非 Config 的 dict 类型，而且字段中没有 dict 类型。
    - 直接根据 `copy_mode = deep/hybrid/shallow` 来选择是 `*<——deepcopy(*)/copy(*)/*`
- （2） `dct` 为 Config 类型，`items()` 字段中没有 dict 类型，`__dict__` 指向的域中的 `_reset_dct_` 为 dict 类型。
    - 合并其 `_copy_kwargs_` 和传入的拷贝参数来拷贝 `dict(items())`（对应（1））；
    - 合并其 `_reset_dct_copy_kwargs_` 和传入的拷贝参数来拷贝 `dict(_reset_dct_)` （对应（1）），并替换其 `self.__dict__._reset_dct_`；
    - 对于 `self.__dict__.` 中非 `_reset_dct_` 的其余字段，浅拷贝。

然后根据最简单的情况就可以构成完整的情况。

#### 函数 `merge_dicts`
```python
def merge_dicts(*dicts: InConfigLikeT,
                to_dict: bool = True,
                copy_mode: tp.Optional[str] = 'shallow',
                nested: bool = True,
                same_keys: bool = False) -> OutConfigLikeT:
    if to_dict:
        dicts = tuple([convert_to_dict(dct, nested=nested) for dct in dicts])
    if copy_mode is not None:
        if not to_dict or copy_mode != 'shallow':
            dicts = tuple([copy_dict(dct, copy_mode=copy_mode, nested=nested) for dct in dicts])
    x, y = dicts[0], dicts[1]
    should_update = True
    if x.__class__ is dict and y.__class__ is dict and len(x) == 0:
        x = y
        should_update = False
    if isinstance(x, atomic_dict) or isinstance(y, atomic_dict):
        x = y
        should_update = False
    if should_update:
        update_dict(x, y, nested=nested, force=True, same_keys=same_keys)
    if len(dicts) > 2:
        return merge_dicts(
            x, *dicts[2:],
            to_dict=False,  
            copy_mode=None,  
            nested=nested,
            same_keys=same_keys
        )
    return x
```